## Introduction to LangChain

In [ ]:
!pip install -q langchain-community langchain-nvidia_ai_endpoints faiss-cpu

In [ ]:
# For Colab user
import os
from google.colab import userdata
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

In [ ]:
# download sample data
!git clone https://github.com/manote101/Building-Apps-with-NIM.git

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_core.runnables import RunnableMap, RunnablePassthrough

# 1. Load and split documents
loader = TextLoader("Building-Apps-with-NIM/data/doc1.txt")  # replace with your file
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

# 2. Create embeddings and vectorstore
embeddings = NVIDIAEmbeddings(model="nvidia/llama-3.2-nv-embedqa-1b-v2")
vectorstore = FAISS.from_documents(splits, embeddings)

# 3. Create retriever
retriever = vectorstore.as_retriever()

# 4. Create LLM and prompt
llm = ChatNVIDIA(model="meta/llama-3.2-3b-instruct", temperature=0)

prompt = ChatPromptTemplate.from_template("""
You are a helpful AI assistant. Use the following context to answer the user's question.

Context:
{context}

Question:
{question}

Answer:
""")

# 5. Create a retrieval-augmented chain
retrieval_chain = RunnableMap({
    # "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "context": lambda x: retriever.invoke(x["question"]),  # Use invoke()
    "question": RunnablePassthrough(),  # Pass the question unchanged
}) | prompt | llm

# 6. Call invoke() to run it
result = retrieval_chain.invoke({"question": "บริษัทใดมีการนำ Nomo microservices มาใข้แล้วบ้าง"})

# 7. Print result
print(result.content)
